[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/GP/Planejamento_Agregado/1_PL_basico.ipynb)

## **MGP**

**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**

In [ ]:
#Execute esse bloco caso esteja executando no Google Colab
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

# Otimização com o Pyomo

## Um problema inicial

Resolveremos um problema simples de Otimização Linear com o Pyomo


Um fazendeiro deseja otimizar as plantações de arroz e milho em sua fazenda

- Os lucros obtidos por unidade de área plantada de arroz e milho são respectivamente de R$ 5,00 e R$2,00
- Em temos de unidade de área plantada, arroz e milho consomem respectivamente um total de 1 homem-hora e 2 homens-hora
- A área plantada de arroz não pode ser maior que 3
- A área plantada de milho não pode ser maior que 4
- Existe a disponibilidade de 9 homens-hora

### Modelagem

$$
\text{Max } z = 5x_{1} + 2x_{2}
$$
$$
\begin{equation}
  \begin{array}{rll}
    \text{Sujeito a: } & \\
    & x_{1}  \leq 3 \\
    & x_{2}  \leq 4 \\
    & x_{1} + 2x_{2} \le 9 \\
    & x_{1} \ge 0, x_{2} \ge 0 \\
  \end{array}
\end{equation}
$$


Primeiramente, importamos a biblioteca com o framework de modelagem pyomo

In [ ]:
# Importando o Pyomo
import pyomo.environ as pe 

Vamos chamar nosso primeiro modelo de $M$. 

A criação do modelo é simples, basta atribuílo à uma variável através do método pe.ConcreteModel(), conforme a célula a seguir:

In [ ]:
M = pe.ConcreteModel()

Lembrando que um modelo matemático de otimização linear possui:
- um conjunto de variáveis;
- um conjunto de restrições.
- uma função objetivo.


Iniciaremos criando as nossas variáveis. Para isso, basta utilizar o método *pe.Var*.

In [ ]:
# Criando variáveis 
x1 = M.x1 = pe.Var(domain=pe.NonNegativeReals) # unidades de área plantada de arroz
x2 = M.x2 = pe.Var(domain=pe.NonNegativeReals) # unidades de área plantada de milho

Para criar restrições, podemos utilizar o método pe.Constraint(). 

Vejamos como ficam as restrições para esse problema simples:

In [ ]:
#Restrições
M.R1 = pe.Constraint(rule= x1 <= 3)
M.R2 = pe.Constraint (rule= x2 <= 4)
M.R3 = pe.Constraint (rule = x1 + 2 * x2 <= 9)

Agora vamos criar a Função Objetivo:

In [ ]:
# Função objetivo
z = M.z = pe.Objective (rule = 5 * x1 + 2 * x2, sense = pe.maximize)


Para resolver o modelo, basta chamar um solver específico.

Neste caso, utilizaremos o solver **gurobi**.

In [ ]:
# Agora é só rodar o solver
pe.SolverFactory('glpk').solve(M)

Imprimindo os resultados na tela:

In [ ]:
print('x1 = ', pe.value(x1))
print('x_2 = ', pe.value(x2))
print('Função objetivo = ', pe.value(z))

## Usando conjuntos


Em problemas mais extensos, pode ser interessante utilizar conjuntos e parâmetros no Pyomo.

Vejamos como ficaria.

In [ ]:
M=pe.ConcreteModel()

# Conjunto
M.tipos = pe.Set(initialize=["arroz", "milho"])

#Parâmetros
M.lucro_unitario = pe.Param(M.tipos, initialize={"arroz": 5, "milho": 2})
M.area_max = pe.Param(M.tipos, initialize={"arroz": 3, "milho": 4} )
M.homem_hora = pe.Param(M.tipos, initialize={"arroz": 1, "milho": 2})
M.hh_capacidade = pe.Param(initialize=9)

#Variáveis
M.x = pe.Var(M.tipos, domain=pe.NonNegativeReals)

#Restrições de área plantada
M.R_area = pe.ConstraintList() 
for i in M.tipos:
    M.R_area.add(M.x[i]<=M.area_max[i])

#Restrições de homem-hora
M.R_hh = pe.Constraint (rule = sum(M.x[i]*M.homem_hora[i] for i in M.tipos) <= M.hh_capacidade)


# Função Objetivo
M.z = pe.Objective(expr = sum(M.x[i] * M.lucro_unitario[i] for i in M.tipos), sense = pe.maximize)

In [ ]:
# Agora é só rodar o solver
pe.SolverFactory('glpk').solve(M)

for i in M.tipos:
    print(f'x[{i}] = {pe.value(M.x[i])}')

print(f'z = {pe.value(M.z)}')

## Exercício 1 - Mix de Produção


Um fabricante de móveis produz mesas e cadeiras. O processo envolve usinagem, lixamento e montagem das peças para confecção das mesas e cadeiras. Leva 5 horas para usinar as peças para uma mesa, 4 horas para lixar as peças e 3 horas para montar uma mesa. Uma cadeira requer 2 horas para usinar as peças, 3 horas para lixar as peças e 4 horas para montar uma cadeira. São 270 horas para usinagem das peças, 250 horas para lixamento das peças e 200 horas para montagem. Se o lucro de uma mesa é de 100 reais e de uma cadeira de 60 reais, quantas mesas e cadeiras o fabricante deve produzir para maximizar o lucro total? E se houver um requisito adicional para produzir quatro cadeiras para cada mesa?


## Exercício 2 - Problema da Dieta

Um nutricionista esportivo está planejando um cardápio alimentar composto de três alimentos principais A, B e C. Cada grama de A contém 3 unidades de proteína, 2 unidades de carboidratos e 4 unidades de gordura. Cada grama de B contém 1, 3 e 2 unidades, respectivamente, e cada grama de C contém 3, 1 e 4 unidades, respectivamente. O nutricionista deseja que a refeição forneça pelo menos 440 unidades de gordura, pelo menos 150 unidades de carboidrato e pelo menos 320 unidades de proteína. Se 1 quilo de A custa 15,60, 1 quilo de B custa 18,90 e 1 quilo de C custa 12,70, quantos gramas de cada alimento devem ser servidos para minimizar o custo da refeição e, ao mesmo tempo, satisfazer as necessidades do nutricionista?


## Exercício 3 - Planejamento de um Evento

O organizador de uma conferência deve fornecer almoços para os cinco dias de uma conferência. O organizador se depara com o problema de decidir sobre o fornecimento diário de guardanapos limpos. Ele sabe que os números de participantes da conferência que precisam almoçar nos cinco dias são 130, 220, 180, 120 e 100. Suas alternativas são: comprar novos guardanapos a 25 centavos cada. Enviar guardanapos usados para a lavandaria onde podem receber um serviço de 48 horas a 10 centavos o guardanapo, ou um serviço de 24 horas a 15 centavos o guardanapo. Considere que esta conferência é uma atividade única, não será repetida no próximo mês/ano. Como o organizador deve fornecer guardanapos para minimizar o custo total?


## Exercício 4 

Uma refinaria produz 3 tipos de gasolina: verde, azul e comum. Cada tipo requer gasolina pura, octana e aditivo, que são disponíveis nas quantidades 9.600.000, 4.800.000 e 2.200.000 litros por semana, respectivamente. As especificações de cada tipo são:

- Um litro de gasolina verde requer 0,22 litro de gasolina pura, 0,50 litro de octana e 0,28 litro de aditivo; 
- Um litro de gasolina azul requer 0,52 litro de gasolina pura, 0,34 litro de octana e 0,14 litro de aditivo; 
- Um litro de gasolina comum requer 0,74 litro de gasolina pura, 0,20 litro de octana e 0,06 litro de aditivo. 

A regra de produção é baseada na demanda de mercado. O planejamento da refinaria estipulou que a quantidade de gasolina comum deve ser **no mínimo** igual a 16 vezes a quantidade de gasolina verde. 

Além disso, foi decidido que a quantidade de gasolina azul seja no máximo igual a 600.000 litros por semana. 

A empresa sabe que cada litro de gasolina verde, azul e comum dá uma margem de contribuição para o lucro de 0,30; 0,25 e 0,20 respectivamente, e seu objetivo é determinar o programa de produção que maximize a margem total de contribuição para o lucro.

1. Apresente um modelo matemático para o problema. Use uma linguagem de modelagem em conjunto com um solver para resolver o problema.

2. Modele e resolva com o pyomo.
